In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 25 messages


[[None, None, None, None, None, None, None], None]

In [3]:
# Declare target name
target_name='alfCMi'

In [9]:
focus_offset=0.01335811-0.02082226-0.00846874
x_offset= -0.36901706999999995-0.59948188+0.72517719 
y_offset=0.07420126000000007-0.59666054+0.48126179 
expTime=60.

In [10]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [11]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=0.8
1580021629-alfCMi_intra-0-1
1580021695-alfCMi_extra-0-1
['1580021629-alfCMi_intra-0-1.fits'], ['1580021695-alfCMi_extra-0-1.fits']
-99.13876385199137


In [13]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

/home/saluser/develop/cwfs/python/data/1580021629-alfCMi_intra-0-1 (1).fits /home/saluser/develop/cwfs/python/data/1580021695-alfCMi_extra-0-1 (1).fits


In [5]:
await cam.cmd_stopLiveView.start()

In [6]:
await cam.cmd_startLiveView.set_start(expTime=0.5)